# Kospi 200 close price LSTM prediction using OHLC

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from keras.models import Sequential
from keras.layers import Dense, LSTM ,Dropout

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


## data load

In [2]:
data = pd.read_csv("stock data/kospi.csv")
data.head()

,date,open,high,low,close
0,2019-12-30,2202.62,2206.36,2188.10,2203.60
1,2019-12-27,2183.70,2215.55,2177.40,2204.21
2,2019-12-26,2192.22,2198.01,2183.90,2197.93
3,2019-12-24,2206.23,2206.23,2187.58,2190.08
4,2019-12-23,2208.22,2209.20,2196.43,2203.71


### reshape to [-1,1]


In [3]:
price_data=data[['open', 'high', 'low', 'close']]

price_data_open=price_data['open'].as_matrix().reshape(-1, 1)
price_data_high=price_data['high'].as_matrix().reshape(-1, 1)
price_data_low=price_data['low'].as_matrix().reshape(-1, 1)
price_data_close=price_data['close'].as_matrix().reshape(-1, 1)

sc_open = MinMaxScaler()
sc_high = MinMaxScaler()
sc_low = MinMaxScaler()
sc_close = MinMaxScaler()

price_data['open'] = sc_open.fit_transform(price_data_open)
price_data['high'] = sc_high.fit_transform(price_data_high)
price_data['low'] = sc_low.fit_transform(price_data_low)
price_data['close'] = sc_close.fit_transform(price_data_close)

price_data=pd.DataFrame(price_data)
price_data.columns=['open','high','low','close']

C:\Users\wsm26\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\wsm26\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\wsm26\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
C:\Users\wsm26\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\wsm26\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [4]:
price_data.head()

,open,high,low,close
0,0.831907,0.826976,0.827091,0.829785
1,0.823706,0.830944,0.822460,0.830048
2,0.827399,0.823370,0.825273,0.827339
3,0.833472,0.826920,0.826866,0.823953
4,0.834335,0.828202,0.830697,0.829833


### y value include

this model will predict 5 days later with 5 days price imformation
- -> example 1,2,3,4,5 day kospi information -> predict 10 day kospi close

In [5]:
data_close = price_data[["close"]]
data_close = data_close[10:].reset_index()[["close"]]
data_close = data_close.rename(columns = {"close" : "y"})

In [6]:
result = pd.concat([price_data,data_close], axis=1)
result = result[:-10]
result.head()

,open,high,low,close,y
0,0.831907,0.826976,0.827091,0.829785,0.815399
1,0.823706,0.830944,0.822460,0.830048,0.801207
2,0.827399,0.823370,0.825273,0.827339,0.787520
3,0.833472,0.826920,0.826866,0.823953,0.784233
4,0.834335,0.828202,0.830697,0.829833,0.780199


In [7]:
y=result['y']

### to make group using 5 days kospi

In [8]:
len(result)

7933

In [9]:
d_matrix=price_data.as_matrix()

p_result =[]
for i in range(len(price_data)-10):
    for j in range(5):
        p_result.append(d_matrix[i+j])

C:\Users\wsm26\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [10]:
p_result=np.array(p_result)
p_result=p_result.reshape((len(d_matrix)-10,5,4))
p_result

array([[[0.83190724, 0.82697564, 0.8270914 , 0.82978531],
        [0.82370611, 0.83094353, 0.82245972, 0.83004844],
        [0.82739922, 0.82337042, 0.82527336, 0.82733943],
        [0.83347204, 0.82691951, 0.8268663 , 0.82395317],
        [0.83433463, 0.82820184, 0.83069718, 0.82983276]],

       [[0.82370611, 0.83094353, 0.82245972, 0.83004844],
        [0.82739922, 0.82337042, 0.82527336, 0.82733943],
        [0.83347204, 0.82691951, 0.8268663 , 0.82395317],
        [0.83433463, 0.82820184, 0.83069718, 0.82983276],
        [0.83318596, 0.82717425, 0.8299916 , 0.8300355 ]],

       [[0.82739922, 0.82337042, 0.82527336, 0.82733943],
        [0.83347204, 0.82691951, 0.8268663 , 0.82395317],
        [0.83433463, 0.82820184, 0.83069718, 0.82983276],
        [0.83318596, 0.82717425, 0.8299916 , 0.8300355 ],
        [0.83052016, 0.82627186, 0.82643777, 0.82674845]],

       ...,

       [[0.27292154, 0.26856901, 0.27196149, 0.26991748],
        [0.27451669, 0.2710171 , 0.27564952, 0.273420

In [11]:
p_result[0]

array([[0.83190724, 0.82697564, 0.8270914 , 0.82978531],
       [0.82370611, 0.83094353, 0.82245972, 0.83004844],
       [0.82739922, 0.82337042, 0.82527336, 0.82733943],
       [0.83347204, 0.82691951, 0.8268663 , 0.82395317],
       [0.83433463, 0.82820184, 0.83069718, 0.82983276]])

### make train, test set

In [12]:
len(result)-10

7923

In [13]:
int(969*0.9)

872

In [14]:
X_train,X_test = p_result[:int(len(p_result)*0.9)],p_result[int(len(p_result)*0.9):]
y_train,y_test = y[:int(len(p_result)*0.9)],y[int(len(p_result)*0.9):]

In [15]:
X_train

array([[[0.83190724, 0.82697564, 0.8270914 , 0.82978531],
        [0.82370611, 0.83094353, 0.82245972, 0.83004844],
        [0.82739922, 0.82337042, 0.82527336, 0.82733943],
        [0.83347204, 0.82691951, 0.8268663 , 0.82395317],
        [0.83433463, 0.82820184, 0.83069718, 0.82983276]],

       [[0.82370611, 0.83094353, 0.82245972, 0.83004844],
        [0.82739922, 0.82337042, 0.82527336, 0.82733943],
        [0.83347204, 0.82691951, 0.8268663 , 0.82395317],
        [0.83433463, 0.82820184, 0.83069718, 0.82983276],
        [0.83318596, 0.82717425, 0.8299916 , 0.8300355 ]],

       [[0.82739922, 0.82337042, 0.82527336, 0.82733943],
        [0.83347204, 0.82691951, 0.8268663 , 0.82395317],
        [0.83433463, 0.82820184, 0.83069718, 0.82983276],
        [0.83318596, 0.82717425, 0.8299916 , 0.8300355 ],
        [0.83052016, 0.82627186, 0.82643777, 0.82674845]],

       ...,

       [[0.13500217, 0.13190334, 0.13424928, 0.13265522],
        [0.13725184, 0.13395852, 0.13712784, 0.135717

In [16]:
X_test

array([[[0.14357174, 0.14025362, 0.14107992, 0.13947951],
        [0.14653663, 0.14363863, 0.14478525, 0.14315048],
        [0.14602948, 0.14517139, 0.14844298, 0.1468085 ],
        [0.14547464, 0.14180796, 0.14513588, 0.14448773],
        [0.14440399, 0.14446762, 0.14681973, 0.14682576]],

       [[0.14653663, 0.14363863, 0.14478525, 0.14315048],
        [0.14602948, 0.14517139, 0.14844298, 0.1468085 ],
        [0.14547464, 0.14180796, 0.14513588, 0.14448773],
        [0.14440399, 0.14446762, 0.14681973, 0.14682576],
        [0.14351539, 0.14262399, 0.14473331, 0.14492772]],

       [[0.14602948, 0.14517139, 0.14844298, 0.1468085 ],
        [0.14547464, 0.14180796, 0.14513588, 0.14448773],
        [0.14440399, 0.14446762, 0.14681973, 0.14682576],
        [0.14351539, 0.14262399, 0.14473331, 0.14492772],
        [0.1373212 , 0.1389022 , 0.13920127, 0.14352577]],

       ...,

       [[0.27292154, 0.26856901, 0.27196149, 0.26991748],
        [0.27451669, 0.2710171 , 0.27564952, 0.273420

In [17]:
y_train

0       0.815399
1       0.801207
2       0.787520
3       0.784233
4       0.780199
          ...   
7134    0.144928
7135    0.143526
7136    0.136848
7137    0.141511
7138    0.135830
Name: y, Length: 7139, dtype: float64

In [18]:
y_test[-20:]

7913    0.289480
7914    0.281181
7915    0.275439
7916    0.269974
7917    0.268805
7918    0.268257
7919    0.269516
7920    0.269917
7921    0.273420
7922    0.273563
7923    0.273528
7924    0.274818
7925    0.273865
7926    0.270081
7927    0.267735
7928    0.266419
7929    0.262882
7930    0.263481
7931    0.264965
7932    0.263559
Name: y, dtype: float64

### model

In [19]:
model = Sequential()

model.add(LSTM(units=100, input_shape=(5,4)))
model.add(Dense(units=1))
model.add(Dropout(0.1))

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               42000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1)                 0         
Total params: 42,101
Trainable params: 42,101
Non-trainable params: 0
_________________________________________________________________


### model train

In [21]:
model.compile(optimizer='rmsprop', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=12)

Epoch 1/100
7139/7139 [==============================] - 4s 591us/step - loss: 0.0257
Epoch 2/100
7139/7139 [==============================] - 4s 563us/step - loss: 0.0285
Epoch 3/100
7139/7139 [==============================] - 4s 508us/step - loss: 0.0265
Epoch 4/100
7139/7139 [==============================] - 3s 453us/step - loss: 0.0274
Epoch 5/100
7139/7139 [==============================] - 3s 459us/step - loss: 0.0247
Epoch 6/100
7139/7139 [==============================] - 4s 515us/step - loss: 0.0259
Epoch 7/100
7139/7139 [==============================] - 4s 499us/step - loss: 0.0271
Epoch 8/100
7139/7139 [==============================] - 4s 572us/step - loss: 0.0266
Epoch 9/100
7139/7139 [==============================] - 4s 517us/step - loss: 0.0262
Epoch 10/100
7139/7139 [==============================] - 3s 481us/step - loss: 0.0277
Epoch 11/100
7139/7139 [==============================] - 4s 522us/step - loss: 0.0271
Epoch 12/100
7139/7139 [============================

7139/7139 [==============================] - 3s 444us/step - loss: 0.0270
Epoch 96/100
7139/7139 [==============================] - 4s 535us/step - loss: 0.0260
Epoch 97/100
7139/7139 [==============================] - 4s 544us/step - loss: 0.0268
Epoch 98/100
7139/7139 [==============================] - 4s 566us/step - loss: 0.0263
Epoch 99/100
7139/7139 [==============================] - 4s 493us/step - loss: 0.0259
Epoch 100/100
7139/7139 [==============================] - 3s 436us/step - loss: 0.0250


### accuracy

In [22]:
predicted = model.predict(X_test)
predicted.shape , y_test.shape
RMSE = mean_squared_error(y_test, predicted)**0.5    
print("RMSE =" ,RMSE)
error = y_test.values - predicted.reshape(1,-1)
print("std ={}" .format(np.std(error)))

RMSE = 0.02553430596902424
std =0.01148330601951726
